# Load dataset

In [1]:
from datasets import load_from_disk
train = load_from_disk("train.hf")
test = load_from_disk("test.hf")

In [2]:
train = train.shuffle(seed=42)

Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\train.hf\cache-aab69560c24eae2b.arrow


In [3]:
test = test.shuffle(seed=42)

Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\test.hf\cache-6c7c52e5435094d8.arrow


# Processing Bert without cleaning

In [5]:
from transformers import AutoTokenizer
from pathlib import Path
bert_model_name = "HooshvareLab/bert-fa-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-base-uncased")

OSError: HooshvareLab/bert-fa-base-uncased does not appear to have a file named config.json. Checkout 'https://huggingface.co/HooshvareLab/bert-fa-base-uncased/None' for available files.

In [8]:
text = " ".join(train["words"][0])
bert_tokens = bert_tokenizer(text).tokens()
bert_tokens

NameError: name 'bert_tokenizer' is not defined

In [ ]:
" ".join(train["words"][0])

## Create a custom model for token classification

In [ ]:
from transformers import BertModel, BertConfig

from pathlib import Path

bert_model = BertModel.from_pretrained("HooshvareLab/bert-fa-base-uncased",cache_dir=Path.cwd())
bert_config = BertConfig.from_pretrained("HooshvareLab/bert-fa-base-uncased",cache_dir=Path.cwd())

In [ ]:
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.bert.modeling_bert import BertPreTrainedModel
from transformers.models.bert.modeling_bert import BertModel
from torch import nn
class ParsBertForTokenClassification(BertPreTrainedModel):
    config_class = bert_config
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # Load model body
        self.parsbert = BertModel(config)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Load and initialize weights
        self.init_weights()
    
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        
        # Use model body to get encoder representations
        outputs = self.parsbert(input_ids, attention_mask=attention_mask,
        token_type_ids=token_type_ids, **kwargs)
        
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        # Return model output object
        return TokenClassifierOutput(loss=loss, logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions)
        

In [ ]:
ner_tags = train["ner"]
ner_tag_names = set(tag for tags in ner_tags for tag in tags)
                
index2tag = {idx: tag for idx, tag in enumerate(ner_tag_names)}
tag2index = {tag: idx for idx, tag in enumerate(ner_tag_names)}

In [ ]:
from transformers import AutoConfig

bert_config = AutoConfig.from_pretrained(bert_model_name,num_labels=4,
                                            id2label=index2tag, label2id=tag2index)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parsbert_model = (ParsBertForTokenClassification
                  .from_pretrained(bert_model_name, config=bert_config,cache_dir=Path.cwd())
                  .to(device))

In [ ]:
import pandas as pd
en_text = "Jack Sparrow loves New York!"
input_ids = bert_tokenizer.encode(en_text, return_tensors="pt")
bert_tokens = bert_tokenizer(en_text).tokens()
pd.DataFrame([bert_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
per_text = " ".join(train['words'][0])
input_ids = bert_tokenizer.encode(per_text, return_tensors="pt")
bert_tokens = bert_tokenizer(per_text).tokens()
pd.DataFrame([bert_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

In [ ]:
outputs = parsbert_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"Number of tokens in sequence: {len(bert_tokens)}")
print(f"Shape of outputs: {outputs.shape}")

In [ ]:
def tag_text(text, tags, model, tokenizer):
    tags = list(ner_tag_names)
    
    # Get tokens with special characters
    tokens = bert_tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = bert_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(inputs)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [tags[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = bert_tokenizer(examples["words"], truncation=True,
    is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append('-100')
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels,batched=True,
                      remove_columns=['id','words','lang','ner'])

In [ ]:
train_encoded = encode_panx_dataset(train)
test_encoded = encode_panx_dataset(test)

In [ ]:
train_encoded, test_encoded

In [ ]:
def align_predictions(predictions, label_ids):
        preds = np.argmax(predictions, axis=2)
        batch_size, seq_len = preds.shape
        labels_list, preds_list = [], []
        for batch_idx in range(batch_size):
            example_labels, example_preds = [], []
            for seq_idx in range(seq_len):
                # Ignore label IDs = -100
                if label_ids[batch_idx, seq_idx] != -100:
                    example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                    example_preds.append(index2tag[preds[batch_idx][seq_idx]])
            labels_list.append(example_labels)
            preds_list.append(example_preds)
        return preds_list, labels_list

In [ ]:
from transformers import TrainingArguments

num_epochs = 10
batch_size = 32
logging_steps = len(train_encoded)
bert_model_name = "HooshvareLab/bert-fa-base-uncased"

training_args = TrainingArguments(
    output_dir="output", log_level="error", num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=False)



In [ ]:
def model_init():
    return (ParsBertForTokenClassification
                  .from_pretrained(bert_model_name, config=bert_config,cache_dir=Path.cwd())
                  .to(device))

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(bert_tokenizer)

In [ ]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
    eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [ ]:
from transformers import Trainer
trainer = Trainer(model_init=model_init, args=training_args,
                    data_collator=data_collator, compute_metrics=compute_metrics,
                    train_dataset=train_encoded,
                    eval_dataset=test_encoded,
                    tokenizer=bert_tokenizer)

In [ ]:
trainer.train()

In [ ]:
str(Path.cwd())

In [ ]:
fr"{str(Path.cwd())}\ParsBert-fa-en-ner"